In [ ]:
pip install pydantic email-validator pydantic-settings jupyter

### 1. Validation without Pydantic

In [1]:
class Product:
    def __init__(self, id: int, name: str, price: float):
        if not isinstance(id, int):
            raise TypeError("ID must be an integer")
        if not isinstance(name, str):
            raise TypeError("Name must be a string")
        if not isinstance(price, (int, float)):
            raise TypeError("Price must be a number")
        self.id = id
        self.name = name
        self.price = price

class Order:
    def __init__(self, order_id: int, customer_email: str, products: list):
        if not isinstance(order_id, int):
            raise TypeError("Order ID must be an integer")
        if not isinstance(customer_email, str):
            raise TypeError("Customer email must be a string")
        if not all(isinstance(p, Product) for p in products):
            raise TypeError("All items in products must be Product objects")
        self.order_id = order_id
        self.customer_email = customer_email
        self.products = products

In [2]:
p1 = Product(id=3, name="Boots", price=42.50)
p2 = Product(id=4, name="Shoe", price=32.0)

o = Order(order_id=3322, customer_email="asdf@dataninja.xyz", products=[p1, p2])


### 2. Validation with Pydantic

In [ ]:
from pydantic import BaseModel

class Product(BaseModel):
    id: int
    name: str
    price: float

p = Product(id=33, name="Radio", price=120.0)

print(p)

### 3. Email validaiton

In [22]:
from pydantic import EmailStr, BaseModel
from typing import List, Optional

class Order(BaseModel):
    order_id: int
    customer_email: EmailStr
    products: Optional[List[Product]] = []
    delivered: bool = False

In [23]:
o = Order(order_id=3322, customer_email="asdf@dataninja.xyz")

In [ ]:
print(o.model_dump_json())

### 4. Regex validation

In [25]:
from pydantic import Field

class Product(BaseModel):
    id: int
    name: str = Field(..., pattern=r"^[a-zA-Z0-9-\s]+$")
    price: float

In [ ]:
product_data = {"id": 1, "name": "Laptop!", "price": 1200.00}

try:
    p = Product(**product_data)
    print(p.model_dump_json(indent=2))
except Exception as e:
    print(f"Error creating order: {e}")

### 5. Nested model validation

In [1]:
from pydantic import BaseModel, Field, EmailStr
from typing import List, Optional

class Product(BaseModel):
    id: int
    name: str = Field(..., pattern=r"^[a-zA-Z0-9\s]+$")
    price: float

class Order(BaseModel):
    order_id: int
    customer_email: EmailStr
    products: List[Product]
    delivery_address: Optional[str] = None

In [ ]:
product_data_1 = {"id": 1, "name": "Laptop", "price": 1200.00}
product_data_2 = {"id": 2, "name": "Mouse", "price": 25.50}

order_data = {
    "order_id": 101,
    "customer_email": "jane.doe@example.com",
    "products": [product_data_1, product_data_2]
}

# Filling with **kwargs!
try:
    order = Order(**order_data)
    print("\nOrder created successfully from dictionary:")
    print(order.model_dump_json(indent=2))
except Exception as e:
    print(f"Error creating order: {e}")

### 6. Computed fields

In [29]:
from pydantic import BaseModel, Field, EmailStr
from typing import List, Optional

class Product(BaseModel):
    id: int
    name: str = Field(..., pattern=r"^[a-zA-Z0-9\s]+$")
    price: float

class Order(BaseModel):
    order_id: int
    customer_email: EmailStr
    products: Optional[List[Product]] = []
    delivery_address: Optional[str] = None

    # Computed field
    @property
    def total_price(self) -> float:
        return sum(product.price for product in self.products)

In [ ]:
product_data_1 = {"id": 1, "name": "Laptop", "price": 1200.00}
product_data_2 = {"id": 2, "name": "Mouse", "price": 25.50}

order_data = {
    "order_id": 101,
    "customer_email": "jane.doe@example.com",
    "products": [product_data_1, product_data_2]
}

try:
    order = Order(**order_data)
    print(order.model_dump_json(indent=2))
    print(f"Total order price: ${order.total_price:.2f}")
except Exception as e:
    print(f"Error creating order: {e}")

### 7. Custom Field Validators

In [ ]:
from pydantic import BaseModel, Field, field_validator

class Product(BaseModel):
    id: int
    name: str = Field(..., pattern=r"^[a-zA-Z0-9\s]+$")
    price: float

    @field_validator("name")
    @classmethod
    def name_must_contain_space(cls, v: str) -> str:
        if ' ' not in v:
            raise ValueError("Product name must contain a space")
        return v

p = Product(id=43, name="asdf asdf", price=454)
print(p.model_dump_json())

### 8. Filling Pydantic models from JSON directly

In [ ]:
import json
from pydantic import ValidationError

json_data = """
{
    "order_id": 102,
    "customer_email": "bob.smith@test.org",
    "products": [
        {"id": 4, "name": "Keyboard", "price": 75.00},
        {"id": 5, "name": "Monitor", "price": 300.00}
    ],
    "delivery_address": "123 Main St, Anytown"
}
"""

try:
    order_from_json = Order.model_validate_json(json_data) # Use model_validate_json for Pydantic v2+
    print("Order created successfully from JSON:")
    print(order_from_json.model_dump_json(indent=2))
except ValidationError as e:
    print(f"Error parsing JSON: {e}")


### 9. Strict mode

In [ ]:
from pydantic import BaseModel, StrictInt, StrictFloat, ValidationError

class StrictProduct(BaseModel):
    id: StrictInt
    price: StrictFloat

try:
    strict_product = StrictProduct(id="1", price=100.0)
except ValidationError as e:
    print(f"Error in strict mode (id): {e}")

try:
    strict_product_2 = StrictProduct(id=2, price=100)
except ValidationError as e:
    print(f"Error in strict mode (price): {e}")

# This will pass
strict_product_ok = StrictProduct(id=3, price=100.0)
print("Strict Product (OK):", strict_product_ok)

### 10. Pydantic Settings - parsing .env files

In [ ]:
import os
from pydantic_settings import BaseSettings, SettingsConfigDict

class AppSettings(BaseSettings):
    model_config = SettingsConfigDict(env_file='.env', extra='ignore')
    database_url: str
    api_key: str
    debug_mode: bool = False
    workers: int = 1

try:
    settings = AppSettings()
    print(f"Database URL: {settings.database_url}")
    print(f"API Key: {settings.api_key}")
    print(f"Debug Mode: {settings.debug_mode}")
    print(f"Workers: {settings.workers}")
except Exception as e:
    print(f"Error loading settings: {e}")
